<a href="https://colab.research.google.com/github/gonzalovaldenebro/NaturalLanguageProcessing-Portfolio/blob/main/F6_1_NeuralLanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; align-items: flex-start;">
  <div>
      <h1>CS 195: Natural Language Processing</h1>
      <h2>Neural Language Modeling</h2>
      </br>
    <a href="https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
  </div>
  <div style="margin-left: 20px;">
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/dalle_neural_net_viz.png?raw=1" width="500" style="display: block;">
  </div>
</div>


**Cover Illustration:** generated by Dall E using the ChatGPT 4 interface, prompted for a visualization of the network used in the code below. *That's not what I meant.*

## Announcement

AI - English Faculty Candidate: Gabriel Ford

Meeting with students: Thursday at 3:30pm in Howard 309

Scholarly Presentation: Friday at 9:00am in Howard ???

## Reference

SLP: Neural Networks and Neural Language Models, Chapter 7 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/7.pdf



In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [2]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Review: Text Classification with Keras

Last time, we saw
* how to do text classification when there are more than 2 classes
    - one hot encoded output layer, one node per class, *softmax* output
    - categorical crossentropy loss
* embedding layer
    - pad sequences to all be same length
    - learn vector for each word representing word semantics
    
<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_text_classification.png?raw=1">
</div>

image source: SLP Fig. 7.11, https://web.stanford.edu/~jurafsky/slp3/7.pdf

*pooling* combines/aggregates all of the embeddings in some way

In [3]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, GlobalMaxPooling1D, GlobalAveragePooling1D
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
#use one of these instead of Flatten() to try a pooling method
#model.add(GlobalMaxPooling1D())
#model.add(GlobalAveragePooling1D())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.3262 - accuracy: 0.8793
Epoch 2/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.1113 - accuracy: 0.9629
Epoch 3/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.0318 - accuracy: 0.9910
Epoch 4/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.0165 - accuracy: 0.9960
Epoch 5/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.0113 - accuracy: 0.9972
Epoch 6/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.0081 - accuracy: 0.9979
Epoch 7/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.0058 - accuracy: 0.9981
Epoch 8/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.0044 - accuracy: 0.9984
Epoch 9/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.0041 - accuracy: 0.9985
Epoch 10/10
238/238 [==============================] - 0s 524us/step - lo

## Neural Language Modeling

**Neural Language Modeling:** predict next word(s) from previous ones - like what we did with Markov models

Like classification, but output is softmax of every possible word in the vocabulary

Often a first step before extending the model to do summarization, translation, dialog, and other NLP tasks

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_language_modeling.png?raw=1">
</div>

image source: SLP Fig. 7.13, https://web.stanford.edu/~jurafsky/slp3/7.pdf

## A Neural Language Model in Keras

Let's start by sampling some data from our news dataset

Then split into a training and testing set

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("ag_news")

data_subset, _ = train_test_split(data["train"]["text"],train_size=5000)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)

Vocabulary size: 18497


## Preparing training examples

We want to take the sequences like

In [5]:
print( train_texts[0] )

[3360, 8378, 16948, 16949, 17, 1, 47, 9, 10, 41, 405, 243, 16950, 187, 89, 785, 16951, 16952, 12, 24, 5426, 130, 4, 1, 2855, 405, 3754, 706]


and slide a window across to predict the next word

In [6]:
print("Use",train_texts[0][0:5],"to predict",train_texts[0][5])
print("Use",train_texts[0][1:6],"to predict",train_texts[0][6])
print("Use",train_texts[0][2:7],"to predict",train_texts[0][7])
print("Use",train_texts[0][3:8],"to predict",train_texts[0][8])
print("etc.")

Use [3360, 8378, 16948, 16949, 17] to predict 1
Use [8378, 16948, 16949, 17, 1] to predict 47
Use [16948, 16949, 17, 1, 47] to predict 9
Use [16949, 17, 1, 47, 9] to predict 10
etc.


## Group Discussion

What data structures (lists, matrixes, etc.) do we need to prepare to make this a classification problem?


I will need a vector that has got one-hot encoded values

## Preparing all of the examples

In [7]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)


print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 137533
First train text: [3360, 8378, 16948, 16949, 17, 1, 47, 9, 10, 41, 405, 243, 16950, 187, 89, 785, 16951, 16952, 12, 24, 5426, 130, 4, 1, 2855, 405, 3754, 706]
Example sequence 0: [3360, 8378, 16948, 16949, 17]  target: 1
Example sequence 1: [8378, 16948, 16949, 17, 1]  target: 47
Example sequence 2: [16948, 16949, 17, 1, 47]  target: 9
Example sequence 3: [16949, 17, 1, 47, 9]  target: 10
Example sequence 4: [17, 1, 47, 9, 10]  target: 41
Example sequence 5: [1, 47, 9, 10, 41]  target: 405


## Padding

Some of the sequences might be really short - so we'll pad them just in case.

In [8]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

## The target output

Since we're making this into a classification problem, the output layer needs to have one node for each word in the vocabulary.

Target values need to be transformed into a one-hot encoded vector

In [9]:

# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [ 3360  8378 16948 16949    17]
target word 0: 1
target word 0 one hot encoded: [0. 1. 0. ... 0. 0. 0.]


## Preparing the test set

We have to do all of those same things for the test set.

**Group Exercise:** Turn this into a function so that you can use it to prepare both the training and test sets.

In [10]:
test_texts = tokenizer.texts_to_sequences(test_data)

# Create the sequences
predictor_sequences_test = []
targets_test = []
for text in test_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences_test.append(curr_predictor_sequence)
        targets_test.append(curr_target)

# Pad sequences to ensure uniform length
predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=sequence_length, padding='pre')

# Convert target to one-hot encoding
target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

## Making the above code a function

In [11]:
def process_data(data,sequence,tokenizer):

  texts = tokenizer.texts_to_sequences(test_data)
  vocabulary_size = len(tokenizer.word_index) + 1

  # Create the sequences
  predictor_sequences = []
  targets = []
  for text in texts:
      for i in range(sequence_length, len(text)):
          # Take the sequence of tokens as input and the next token as target
          curr_target = text[i]
          curr_predictor_sequence = text[i-sequence_length:i]
          predictor_sequences.append(curr_predictor_sequence)
          targets.append(curr_target)

  # Pad sequences to ensure uniform length
  predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

  # Convert target to one-hot encoding
  target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

  return predictor_sequences_padded, target_word_one_hot

predictor_sequences_padded, target_word_one_hot = process_data(train_data, 5, tokenizer)
predictor_sequences_padded_test, target_word_one_hot_test = process_data(test_data,  5, tokenizer)

## Designing the Neural Network

We'll start with a simple network like the one we used for text classification.

In [12]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
1066/1066 [==============================] - 13s 12ms/step - loss: 7.9272 - accuracy: 0.0478 - val_loss: 7.1475 - val_accuracy: 0.0536
Epoch 2/10
1066/1066 [==============================] - 13s 12ms/step - loss: 7.0124 - accuracy: 0.0682 - val_loss: 6.5881 - val_accuracy: 0.0828
Epoch 3/10
1066/1066 [==============================] - 14s 13ms/step - loss: 6.5280 - accuracy: 0.0897 - val_loss: 6.1298 - val_accuracy: 0.1104
Epoch 4/10
1066/1066 [==============================] - 16s 15ms/step - loss: 6.0881 - accuracy: 0.1205 - val_loss: 5.6770 - val_accuracy: 0.1518
Epoch 5/10
1066/1066 [==============================] - 16s 15ms/step - loss: 5.6302 - accuracy: 0.1565 - val_loss: 5.1736 - val_accuracy: 0.1981
Epoch 6/10
1066/1066 [==============================] - 16s 15ms/step - loss: 5.1139 - accuracy: 0.2053 - val_loss: 4.6100 - val_accuracy: 0.2523
Epoch 7/10
1066/1066 [==============================] - 17s 16ms/step - loss: 4.5397 - accuracy: 0.2607 - val_loss: 3.9712 -

## Testing the final model on the test set

In [13]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

1066/1066 [==============================] - 3s 3ms/step - loss: 1.7332 - accuracy: 0.6968
Test accuracy: 69.68%


## Text Generation

We can use this model to successively generate words based on previous ones - like our Markov sequence on steroids.

Let's see how this works

In [14]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
print(tokens_list)

tokens_array = np.array(tokens_list)
print(tokens_array)

[[1, 100, 21, 11, 22]]
[[  1 100  21  11  22]]


the model will predict probabilities for each possible word in the output

In [15]:
predicted_probabilities = model.predict(tokens_array,verbose=0)
print(predicted_probabilities)
print("We get a probability for each of the",len(predicted_probabilities[0]),"words")

[[1.3087605e-22 8.5567290e-06 6.3590189e-05 ... 1.3707638e-22
  1.1471919e-22 1.4834443e-22]]
We get a probability for each of the 18497 words


then we could get the word associated with the highest probability

In [16]:
predicted_index = np.argmax(predicted_probabilities)
print("word index:",predicted_index)
predicted_word = tokenizer.index_word[predicted_index]
print("word:",predicted_word)

word index: 94
word: would


or you could generate a random word according the these probabilities (like with did with Markov text generation)

### putting it all together

In [17]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
tokens = tokens_list[0]

for i in range(50):
    curr_seq = tokens[-sequence_length:]
    curr_array = np.array([curr_seq])
    predicted_probabilities = model.predict(curr_array,verbose=0)
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = tokenizer.index_word[predicted_index]
    print(predicted_word+" ",end="")
    tokens.append(predicted_index)


would be investing attacks in a call for the islamic time this week is taken to be sen 000 capacity in his right who does not gone for the release of a quot assumption and rising in new york reuters russian baseball security council has asked that an executive of 

## Applied Exploration

Pick another dataset and get it working with this code
* you will likely need to prepare the text a little differently - do you need to first break it into sentences?
* describe your dataset and what you did to prepare it

Perform a neural language modeling experiment
* experiment with something to try to find a well-performing model
    * sliding window size
    * number of hidden nodes in the network
    * learning rate
* describe what you did and write up an interpretation of your results

In [18]:
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])